In [ ]:
!pip install selenium


In [ ]:
#Cell to get the route name and links
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

#Instantiate the driver
driver = webdriver.Chrome()
driver.maximize_window()
#Store the name and links for the government buses
route_attributes=[]

#Get the bus names and links for 10 government buses
for x in range(1,11):
    driver.get('https://www.redbus.in/')
    driver.implicitly_wait(2)
    #Make a government bus selection
    gov_selector=".rtcBack:nth-child("+str(x)+") .rtcName"
    gov_select=driver.find_element(By.CSS_SELECTOR,gov_selector)
    driver.execute_script("arguments[0].click();", gov_select)
    #current_url=driver.current_url
    #print(current_url)
    
    i=2 # Counter to Traverse through tabs
    while True: # Pagination loop
        try:
            # Find the different routes available in each page
            
            elements = driver.find_elements(By.CLASS_NAME,'route')
    
            
            for element in elements:
                #Get the text (name) and link (href attribute)
                title = element.get_attribute('title')
                link = element.get_attribute('href')
                route_attributes.append({'title':title,'link':link})
            #Go to next tab
            selector=".DC_117_pageTabs:nth-child("+str(i)+")"
            next_page=driver.find_element(By.CSS_SELECTOR, selector)
            driver.execute_script("arguments[0].click();", next_page)
            i+=1
            
        except:
            break
print(len(route_attributes))
print(route_attributes)
driver.quit()

In [ ]:
#Cell to get the bus details for the each link got above 
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

redbus_data=[]

#Get the bus details for all the routes
for route in route_attributes:
    #Instantiate the driver
    driver = webdriver.Chrome()
    route_name=route['title']
    route_link=route['link']
    driver.get(route_link)
    driver.maximize_window()
    time.sleep(2)#adjust according to the pageload time
    
    #expand the government buses section
    for x in range(1,5):
        try:
            driver.implicitly_wait(2)
            gov_bus_selector=".group-data:nth-child("+str(x)+") .button"
            gov_bus_select=driver.find_element(By.CSS_SELECTOR,gov_bus_selector)
            driver.execute_script("arguments[0].click();", gov_bus_select)#Make a government bus selection
        except:
            break


    # Scroll down to the bottom of the page

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for page to load
        time.sleep(2)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    li_elements = driver.find_elements(By.CSS_SELECTOR,'.bus-items li[id]')
    print(len(li_elements),route_name)     
    if bool(li_element) == True:
        for li_element in li_elements:
            try:
                #driver.implicitly_wait(5)
                id=li_element.get_attribute('id')
                bus_name=li_element.find_element(By.CSS_SELECTOR,'.travels').text
                bus_type=li_element.find_element(By.CSS_SELECTOR,'.bus-type').text
                Departing_Time=li_element.find_element(By.CSS_SELECTOR,'.dp-time ').text
                Duration=li_element.find_element(By.CSS_SELECTOR,'.dur').text
                Reaching_Time=li_element.find_element(By.CSS_SELECTOR,'.bp-time').text
                try:
                    Reaching_Day=li_element.find_element(By.CSS_SELECTOR,'.next-day-dp-lbl').text
                except:
                    Reaching_Day=''
                try:
                    Star_Rating=li_element.find_element(By.CSS_SELECTOR,'.rating-sec').text
                except:
                    Star_Rating=''
                Price=li_element.find_element(By.CSS_SELECTOR,'.column-seven .f-bold').text
                Seat_left=li_element.find_element(By.CSS_SELECTOR,'.seat-left').text
                Seat_Availability=Seat_left.split(' ')
                Reaching_Time_Day=Reaching_Time+' '+Reaching_Day
            except Exception as e:
                    print(e)
                    continue     
            redbus_data.append({'route_name':route_name,'route_link':route_link,'id':id,'bus_name':bus_name,
                                'bus_type':bus_type,'Departing_Time':Departing_Time,
                                'Duration':Duration,'Reaching_Time_Day':Reaching_Time_Day,'Star_Rating':Star_Rating,
                                'Price':Price,'Seat_Availability':Seat_Availability[0]})
        #print(len(redbus_data)) #To check the no. of buses extracted for each link
        driver.quit()
print(len(redbus_data)) #size of data extracted from all the links

In [ ]:
# #Writing bus details in a file for backup purpose--optional---
# import pickle
# data=redbus_data
# with open("Redbus_bus_Details.pkl",mode="wb+") as file:
#     pickle.dump(data,file)


In [ ]:
# #Can read bus details data for file if needed--optional---
# with open("Redbus_bus_Details.pkl",mode="rb+") as file:
#     data1=pickle.load(file)
# data1

In [ ]:
# #Writing route details in a file for backup purpose--optional---
# data=route_attributes
# with open("Redbus_route_Details.pkl",mode="wb+") as file:
#     pickle.dump(data,file)

In [ ]:
# #Can read route details data for file if needed--optional---
# with open("Redbus_route_Details.pkl",mode="rb+") as file:
#     data1=pickle.load(file)
# data1

In [ ]:
##################################2.Store data in database
#Install mysql- connector

!pip install mysql-connector-python



In [ ]:
#import and establish connection
import mysql.connector
con= mysql.connector.connect(
        host=<'hostname'>, # include your mysqldb hostname eg:'localhost'
        user=<'username'>, #include your mysqldb username eg:'root'
        password=<'password'> #include your mysqldb password eg:'12345678'
    #, database='capstone'   #needed to connect to database directly once the database is created
)
cursor=con.cursor(buffered=True)
cursor

In [ ]:
#create a database
query='create database Capstone'
cursor.execute(query)

In [ ]:
#To check whether the database(capstone) is created
query='show databases'
cursor.execute(query)
for db in cursor:
    print(db)

In [ ]:
#use the created database
query='use capstone'
cursor.execute(query)

In [ ]:
#To check whether the correct database is selected
query='select database()'
cursor.execute(query)
for db in cursor:
    print(db)

In [ ]:
#Create a table bus_route with the given specification --page 2 data
query='''create table bus_routes(id INT AUTO_INCREMENT PRIMARY KEY,
                                route_name VARCHAR(255),route_link VARCHAR(255),busname VARCHAR(255),
                                bustype VARCHAR(255),departing_time time,duration VARCHAR(255),
                                reaching_time time, star_rating DECIMAL(2,1),price DECIMAL(10,2),seats_available INT)'''
cursor.execute(query)

In [ ]:
#To check the table structure
query="describe bus_routes"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#Create a table route_details with the given specification --page 1 data
query='''create table route_details(id INT AUTO_INCREMENT PRIMARY KEY,
                                route_name VARCHAR(255),route_link VARCHAR(255))'''
cursor.execute(query)

In [ ]:
#To check the table structure
query="describe route_details"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#Insert data into route_details table

query='''insert into route_details(route_name,route_link) values(%s,%s)'''
# Insert data into the table
for item in route_attributes:
    data=(item['title'],item['link'])
    cursor.execute(query,data)

# Commit changes and close the connection
con.commit() 

In [ ]:
#view the route_details table
query='select * from route_details'
cursor.execute(query)
for db in cursor:
    print(db)

In [ ]:
#Insert the extracted values into the table
query='''insert into bus_routes(route_name,route_link,busname,bustype,departing_time,duration,
                                reaching_time, star_rating ,price,seats_available) 
                                values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
# Insert data into the table
for item in redbus_data:
    reaching_time=item['Reaching_Time_Day'].split(' ')
    star_rating=[None if (item['Star_Rating']=='New' or item['Star_Rating']=='') else float(item['Star_Rating'])]
    price=[None if item['Price']=='' else float(item['Price'])]
    seats=[None if item['Seat_Availability']=='' else int(item['Seat_Availability'])]
    data=(item['route_name'],item['route_link'],item['bus_name'],item['bus_type'],
          item['Departing_Time'],item['Duration'],reaching_time[0],star_rating[0],
          price[0],seats[0])
    cursor.execute(query,data)

# Commit changes and close the connection
con.commit()   


In [ ]:
# view the data in the table
query='select * from bus_routes'
cursor.execute(query)
for data in cursor:
    print(data)